# Weekly Merge
Notebook to facilitate the weekly merge with Maddie's Data

In [27]:
import pandas as pd
import numpy as np

In [55]:
# Read in data
input_data = pd.read_csv("input_data_old.csv")
maddie = pd.read_excel("Scoring Workbook March 30.ods", engine='odf')
# maddie = pd.read_excel("Scoring Workbook.ods", engine='odf')

In [56]:
# Tweak Maddie's show names
maddie = maddie.replace({"Bachelor":"The Bachelor"})

In [57]:
maddie.head()

,Show,Contestant,Week 1,Week 2,Week 3 1,Week 32,Week 4,Week 5,Week 6,Week 7 1,Week 7 2,Week 8,Week 9,Week 10
0,The Bachelor,Alayah,0,0,0,0,0,0,0,X,X,0,X,X
1,The Bachelor,Alexa,0,0,0,0,0,0,0,X,X,0,X,X
2,The Bachelor,Deandra,1,0,0,0,0,0,0,X,X,0,X,X
3,The Bachelor,Hannah Ann,1,1,1,1,1,1,1,X,X,5,X,X
4,The Bachelor,Jasmine,0,0,0,0,0,0,0,X,X,0,X,X


In [58]:
# Trim my data (I trust Maddie more)
# input_data.drop(columns=["Week 1", "Week 2", "Score"], inplace=True)
input_data = input_data[["Show", "Name", "file", "PickedBy", "Eliminated"]]

In [59]:
# Merge on show and contenstant
df = pd.merge(input_data, maddie, left_on=["Show", "Name"],
              right_on=['Show', "Contestant"], how='left')

In [60]:
df.Show.unique()

array(['Kids Baking Championship', 'Project Runway', 'Survivor',
       'The Bachelor', 'Top Chef'], dtype=object)

In [61]:
# Fill X and NA with 0
df.fillna(0., inplace=True)
df.replace({"X":0.}, inplace=True)

In [62]:
# Change datatypes for weeks
for col in df.columns:
    if "Week" in col:
        df[col] = df[col].astype(float)

In [63]:
# Get Scores
week_cols = list(filter(lambda x: "Week" in x, list(df.columns)))
df['Score'] = df[week_cols].sum(axis=1)

In [64]:
# Handle eliminated
last_week = week_cols[-1]
df['Eliminated'] = np.where(df[last_week] == 0, 1, 0)

In [65]:
last_week

'Week 10'

In [66]:
df

,Show,Name,file,PickedBy,Eliminated,Contestant,Week 1,Week 2,Week 3 1,Week 32,Week 4,Week 5,Week 6,Week 7 1,Week 7 2,Week 8,Week 9,Week 10,Score
0,Kids Baking Championship,Anthony,kbc_anthony.jpg,Martyna,1,Anthony,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,Kids Baking Championship,Avner,kbc_avner.jpg,Matt,1,Avner,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
2,Kids Baking Championship,Elise,kbc_elise.jpg,Shelley,1,Elise,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Kids Baking Championship,Graysen,kbc_graysen.jpg,Jerry,1,Graysen,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,3.0,0.0,0.0,0.0,10.0
4,Kids Baking Championship,Liam,kbc_liam.jpg,Shelley,1,Liam,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Top Chef,Lee Anne,tc_leeanne.jpg,0,0,Lee Anne,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0
69,Top Chef,Lisa,tc_lisa.jpg,Kenny,0,Lisa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0
70,Top Chef,Melissa,tc_melissa.jpg,Jessica,0,Melissa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0
71,Top Chef,Nini,tc_nini.jpg,Kenny,0,Nini,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0


In [67]:
# Fix for Top Chef (hasn't started)
# df['Eliminated'] = np.where(df.Show == "Top Chef", 0, df.Eliminated)

In [68]:
df.to_csv("input_data.csv")

In [69]:
## Save to JSON
s = df.to_json(orient='records')
f = open("data.js", 'w+')
f.write("var data = " + s + ';')
f.close()